In [ ]:
#Possible usefull dependencies
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import tensorflow.keras.layers as tfkl
import keras

from datetime import datetime
from dateutil.tz import gettz

import scipy
import cv2
import os


## Data Load

Dataset import from kaggle folder.

In [ ]:
dataset_path = "/kaggle/input/dataset-1/Dataset_new_new"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

images1 = np.load(dataset_path + "/s1_micro_images.npz")['images']
labels1 = np.load(dataset_path + "/s1_micro_labels.npz")['labels']
images2 = np.load(dataset_path + "/s1_segre_images.npz")['images']
labels2 = np.load(dataset_path + "/s1_segre_labels.npz")['labels']
images3 = np.load(dataset_path + "/s1_top_images.npz")['images']
labels3 = np.load(dataset_path + "/s1_top_labels.npz")['labels']
images4 = np.load(dataset_path + "/s2_micro_images.npz")['images']
labels4 = np.load(dataset_path + "/s2_micro_labels.npz")['labels']
images5 = np.load(dataset_path + "/s2_segre_images.npz")['images']
labels5 = np.load(dataset_path + "/s2_segre_labels.npz")['labels']
images6 = np.load(dataset_path + "/s2_top_images.npz")['images']
labels6 = np.load(dataset_path + "/s2_top_labels.npz")['labels']
images7 = np.load(dataset_path + "/s4_micro_images.npz")['images']
labels7 = np.load(dataset_path + "/s4_micro_labels.npz")['labels']
images8 = np.load(dataset_path + "/s4_pata_images.npz")['images']
labels8 = np.load(dataset_path + "/s4_pata_labels.npz")['labels']
images9 = np.load(dataset_path + "/s4_wel_images.npz")['images']
labels9 = np.load(dataset_path + "/s4_wel_labels.npz")['labels']

In [ ]:
x_train = np.concatenate((images1, images2, images3, images4, images5, images6, images7, images8, images9))

In [ ]:
y_train = np.concatenate((labels1, labels2, labels3, labels4, labels5, labels6, labels7, labels8, labels9))

In [ ]:
plt.imshow(x_train[327, :, :, 0], cmap="gray")
plt.show()

In [ ]:
plt.imshow(y_train[327, :, :, 0], cmap="gray")
plt.show()

In [ ]:
images10 = np.load(dataset_path + "/s5_count_images.npz")['images']
labels10 = np.load(dataset_path + "/s5_count_labels.npz")['labels']
images11 = np.load(dataset_path + "/s5_ka_images.npz")['images']
labels11 = np.load(dataset_path + "/s5_ka_labels.npz")['labels']

In [ ]:
x_val = np.concatenate((images10, images11))
y_val = np.concatenate((labels10, labels11))

In [ ]:
plt.imshow(x_val[2, :, :, 0], cmap="gray")
plt.show()

In [ ]:
plt.imshow(y_val[2, :, :, 0], cmap="gray")
plt.show()

In [ ]:
images12 = np.load(dataset_path + "/s5_pa_images.npz")['images']
labels12 = np.load(dataset_path + "/s5_pa_labels.npz")['labels']

In [ ]:
x_test = images12
y_test = labels12

In [ ]:
print('x_train shape', x_train.shape)
print('y_train shape', y_train.shape)
print('x_val shape', x_val.shape)
print('y_val shape', y_val.shape)
print('x_test shape', x_test.shape)
print('y_test shape', y_test.shape)

In [ ]:
NUM_CLASSES = 7

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=NUM_CLASSES)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=NUM_CLASSES)

In [ ]:
print('y_train shape', y_train.shape)
print('y_val shape', y_val.shape)
print('y_test shape', y_test.shape)

## Label count
Extraction of mean values of number of pixels in each class in order to set the inital weights.

In [ ]:
label_count = np.zeros([7, 1])

for i in range(np.size(y_train, 3)):
  for j in range(np.size(y_train, 0)):
    label_count[i] = label_count[i] + np.sum(y_train[j, :, :, i])

label_count = label_count/np.size(y_train, 0)
label_count

In [ ]:
label_mean_values = {"BKG": 51575.88414634,
           "UL": 103.51676829,
           "HP": 150.63567073,
           "SP": 101.21493902,
           "TO": 1141.98018293,
           "LL": 548.25304878,
           "HE": 11914.5152439
           }
print(label_mean_values)

In [ ]:
# Compute weights for the weighted loss
import numpy as np

# Calculate the inverse frequency
class_weights = 1.0 / label_count.flatten()

# Normalize the weights (optional but recommended)
normalized_weights = class_weights * NUM_CLASSES / np.sum(class_weights)

print("Class weights:", class_weights)
print("Normalized class weights:", normalized_weights)

#Dataset creation

In [ ]:
BATCH_SIZE = 8

# Create dataset where each item is a tuple (image, image_segmented)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(
    lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.float32))
)

val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).map(
    lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.float32))
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(
    lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.float32))
)

# Shaffle and batch train_ds
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 16)  # mix the data
train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=False)  # create batch
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)  # improve the computational performance

# Batch val_ds
val_ds = val_ds.batch(BATCH_SIZE, drop_remainder=False)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

# Take 1 batch as example
for batch in train_ds.take(1):
    print(batch[0].shape)  # it should be (BATCH_SIZE, H, W, C)
    print(batch[1].shape)  # it should be (BATCH_SIZE, H, W, NUM_CLASSES)

In [ ]:
del x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
pip install keras-cv --upgrade

In [ ]:
seed= 42

In [ ]:
@tf.function
def random_flip(image, label, seed=seed):
    """Consistent random horizontal flip for both image and label."""

    # Determine flip probability
    flip_prob = tf.random.uniform([], seed=seed)
    # print(f"flip_horizontal: {flip_prob}")

    # Image flip
    image = tf.cond(
        flip_prob > 0.5,
        lambda: tf.image.flip_left_right(image),
        lambda: image
    )

    # Segmentation flip (label)
    label = tf.cond(
        flip_prob > 0.5,
        lambda: tf.image.flip_left_right(label),
        lambda: label
    )

    return image, label

In [ ]:
@tf.function
def random_brightness(image, label, max_delta=0.2, seed=seed):
    """Increase or decrease brightness by a random factor."""
    # Determine the probability of modifying brightness
    brightness_prob = tf.random.uniform([], minval=0., maxval=1., seed=seed)
    #print(f"brightenss: {brightness_prob}")

    # Modify brightness
    image = tf.cond(
        brightness_prob > 0.5,
        lambda: tf.image.random_brightness(image, max_delta=max_delta, seed=seed),
        lambda: image
    )

    return image, label

In [ ]:
@tf.function
def random_crop(image, label, crop_size=(56, 112), seed=None):
    """Random crop on image and label"""

    # Determine crop probability
    crop_prob = tf.random.uniform([], minval=0., maxval=1., seed=seed)

    # get image and label shape (batch_size, height, width, channels)
    image_shape = tf.shape(image)
    label_shape = tf.shape(label)

    # Get channel number
    image_channels = image_shape[-1]
    label_channels = label_shape[-1]

    batch_size = image_shape[0]
    target_size = (64, 128)

    # Random crop application with one probability
    image = tf.cond(
        crop_prob > 0.5,
        lambda: tf.image.random_crop(image, size=[batch_size,crop_size[0], crop_size[1], image_channels], seed=seed),
        lambda: image
    )

    label = tf.cond(
        crop_prob > 0.5,
        lambda: tf.image.random_crop(label, size=[batch_size, crop_size[0], crop_size[1], label_channels], seed=seed),
        lambda: label
    )

    image = tf.image.resize(image, target_size)
    label = tf.image.resize(label, target_size)

    return image, label

In [ ]:
@tf.function
def random_contrast(image, label, lower=0.5, upper=1.5, seed=seed):
    """Applies a random contrast adjustment to the image."""

    # Determine the probability of contrast adjustment
    contrast_prob = tf.random.uniform([], minval=0., maxval=1., seed=seed)

    # Modify the contrast
    image = tf.cond(
        contrast_prob > 0.5,
        lambda: tf.image.random_contrast(image, lower=lower, upper=upper, seed=seed),
        lambda: image
    )

    return image, label

In [ ]:
@tf.function
def augment_image(image, label, seed=seed):
    """Apply augmentations."""
    # Horizontal flip
    image, label = random_flip(image, label, seed=seed)

    # Adjust brightness
    image, label = random_brightness(image, label, seed=seed)

    # Contrast distorsion
    image, label = random_contrast(image, label, seed=seed)

    return image, label

In [ ]:
train_ds = train_ds.map(
    lambda x, y: augment_image(x, y, seed=seed),  # before apply augmentation and then  normalization
    num_parallel_calls=tf.data.AUTOTUNE
)

## Preprocessing: sono già normalizzate tra 0 e 1


## Dataset Visualization: da fare se vogliamo



## Loss Function

Cross Entropy:  $\, L_{CE} = -\sum_{c=1}^{C} \frac{1}{N} \sum_{i=1}^{N} g_i^c\,log\,p_i^c \quad$ [[1]](https://www.sciencedirect.com/science/article/pii/S1361841521000815)

where:
* $C$ = n°classes
* $N$ = n°pixels of image (256x256)
* $g_i^c$ = value of the $i^{th}$ pixel of the $c^{th}$ class of the ground truth label
* $p_i^c$ = value of the $i^{th}$ pixel of the $c^{th}$ class of the predicted label

In [ ]:
def cross_entropy(num_classes):
    def loss(y_true, y_pred): #called each iteration (step)

        if len(y_pred.shape) == 4: #2D image
            axis = (1,2)
        elif len(y_pred.shape) == 5: #3D volume
            axis == (1,2,3)

        y_true = tf.cast(y_true, dtype = tf.float32)
        y_pred = tf.cast(y_pred, dtype = tf.float32)

        loss_image = 0.0
        loss_ce = 0.0

        for c in range(num_classes):

            loss_image = tf.math.add(-1*(tf.math.reduce_mean(tf.math.multiply(y_true[...,c], tf.math.log(y_pred[...,c] + 1e-9)), axis = axis)), loss_image)

        loss_ce = tf.math.reduce_mean(loss_image)

        return loss_ce

    return loss

#in model.compile can also be set the default tf.keras.losses.CategoricalCrossentropy()

weighted cross entropy loss


In [ ]:
def weighted_cross_entropy(num_classes, class_weights):
    """
    Weighted categorical cross-entropy loss with class weights for 2D/3D inputs.

    Args:
    - num_classes (int): Number of classes.
    - class_weights (list or array): Weights for each class. Length must match `num_classes`.

    Returns:
    - A callable loss function.
    """
    def loss(y_true, y_pred):
        # Assicurati che y_true e y_pred siano float32
        y_true = tf.cast(y_true, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)

        # Prevenzione di log(0) con epsilon
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        # Calcolo della cross-entropy ponderata per ogni classe
        weighted_loss = 0.0
        for c in range(num_classes):
            weight = class_weights[c]
            class_loss = -y_true[..., c] * tf.math.log(y_pred[..., c])
            weighted_loss += weight * class_loss

        # Media su tutti gli assi spaziali e il batch
        loss_ce = tf.reduce_mean(weighted_loss)
        return loss_ce

    return loss

In [ ]:
import tensorflow.keras.backend as K

def dice_loss(y_true, y_pred, smooth=K.epsilon()):
    """
    Calcola la Dice Loss, che misura la similarità tra le immagini segmentate e quelle vere.

    Args:
    - y_true: Etichette vere (ground truth) in formato one-hot encoded o etichette binarie.
    - y_pred: Predizioni del modello in formato one-hot encoded o probabilità.
    - smooth: Un piccolo valore per evitare la divisione per zero.

    Returns:
    - La Dice Loss.
    """
    # Assicurati che le predizioni siano trattate come probabilità
    #y_pred = K.clip(y_pred, 0, 1)

    # Calcolo della Dice Similarity Coefficient (DSC)
    intersection = K.sum(y_true * y_pred, axis=(1, 2, 3))  # Somma per ogni classe su tutte le immagini
    union = K.sum(y_true, axis=(1, 2, 3)) + K.sum(y_pred, axis=(1, 2, 3))  # Somma delle aree di verità a terra e previste
    dice = (2.0 * intersection + smooth) / (union + smooth)  # Formula della Dice Similarity

    return 1 - dice  # La Dice Loss è 1 meno il DSC (perché vogliamo minimizzare la perdita)

In [ ]:
def combined_loss(num_classes, class_weights, alpha=0.5, beta=0.5, smooth=K.epsilon()):
    """
    Combina la Weighted Cross-Entropy Loss e la Dice Loss in una funzione di perdita composita.

    Args:
    - num_classes (int): Numero di classi.
    - class_weights (list or array): Pesi per ogni classe per la cross-entropy.
    - alpha (float): Ponderazione per la Weighted Cross-Entropy Loss.
    - beta (float): Ponderazione per la Dice Loss.
    - smooth (float): Piccolo valore per evitare la divisione per zero nella Dice Loss.

    Returns:
    - Una funzione di perdita combinata.
    """
    # Recupera le funzioni di perdita individuali
    wce_loss = weighted_cross_entropy(num_classes, class_weights)

    def loss(y_true, y_pred):
        # Calcola le singole perdite
        loss_wce = wce_loss(y_true, y_pred)
        loss_dice = dice_loss(y_true, y_pred, smooth=smooth)

        # Combina le perdite con i pesi
        combined = alpha * loss_wce + beta * loss_dice
        return combined

    return loss

In [ ]:
focal_loss = tf.keras.losses.CategoricalFocalCrossentropy(
    alpha=class_weights,
    gamma=2.0,
    name='categorical_focal_crossentropy'
)

## Metric



Mean Dice: $\, DICE = \frac{1}{C}  \sum_{c = 1}^{C} \frac{2 \lvert S_g^c \cap S_p^c \rvert}{\lvert S_g^c \rvert + \lvert S_p^c \rvert} = \frac{2\,\sum_{c=1}^{C}\sum_{i=1}^{N}g_i^c\,p_i^c}{\sum_{c=1}^{C}\sum_{i=1}^{N}g_i^c \,+\, \sum_{c=1}^{C}\sum_{i=1}^{N}p_i^c  } \quad$ [[3]](https://bmcmedimaging.biomedcentral.com/track/pdf/10.1186/s12880-015-0068-x.pdf)

In [ ]:
class Mean_DICE(keras.metrics.Metric):
    def __init__(self, num_classes, name='Mean_DICE', smooth_factor=1e-9, **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.smooth_factor = smooth_factor

        # Variabili accumulative
        self.total_dice = self.add_weight(name="total_dice", initializer="zeros", dtype=tf.float32)
        self.num_samples = self.add_weight(name="num_samples", initializer="zeros", dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)

        if len(y_pred.shape) == 4:
            axis = (1, 2)
        elif len(y_pred.shape) == 5:
            axis = (1, 2, 3)
        else:
            raise ValueError("y_pred must have 4 or 5 dimensions.")

        dice_classes = 0.0

        for c in range(self.num_classes):
            abs_label = tf.reduce_sum(y_true[..., c], axis=axis)
            abs_pred = tf.reduce_sum(y_pred[..., c], axis=axis)

            MD_batch_denom = abs_label + abs_pred + self.smooth_factor
            MD_batch_num = 2 * tf.reduce_sum(y_true[..., c] * y_pred[..., c], axis=axis) + self.smooth_factor

            dice_image = MD_batch_num / MD_batch_denom
            dice_classes += dice_image

        mean_dice = dice_classes / self.num_classes
        batch_mean_dice = tf.reduce_mean(mean_dice)

        # Aggiornamento delle variabili accumulative
        self.total_dice.assign_add(batch_mean_dice)
        self.num_samples.assign_add(1.0)

    def result(self):
        return self.total_dice / self.num_samples

    def reset_states(self):
        self.total_dice.assign(0.0)
        self.num_samples.assign(0.0)

    def get_config(self):
        config = super().get_config()
        config.update({'num_classes': self.num_classes, 'smooth_factor': self.smooth_factor})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


## Architectures: questa è la rete che ho modificato, poi comunque possiamo usare anche altri paramentri oppure riutilizzare quella vecchia

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfkl

def residual_block(x, n_filters):
  '''implementation of the residual block described in
  https://www.sciencedirect.com/science/article/pii/S0169260721000201?via%3Dihub'''
  shortcut = x

  x = tfkl.BatchNormalization()(x)
  x = tfkl.ReLU()(x)
  x = tfkl.Conv2D(n_filters, 3, padding = 'same', activation = None)(x)

  x = tfkl.BatchNormalization()(x)
  x = tfkl.ReLU()(x)
  x = tfkl.Conv2D(n_filters, 3, padding = 'same', activation = None)(x)

  shortcut = tfkl.Conv2D(n_filters, 1, padding = 'same', activation = None)(shortcut)
  shortcut = tfkl.BatchNormalization()(shortcut)

  out = tfkl.Add()([shortcut, x])
  return out

def decoder_block(x, n_filters, skip, dropout):
  '''implementation of the decoder block described in
  https://www.sciencedirect.com/science/article/pii/S0169260721000201?via%3Dihub'''
  x = tfkl.Conv2DTranspose(n_filters, 1, strides=(2, 2), padding='same')(x)
  x = tfkl.BatchNormalization()(x)

  x = tfkl.Concatenate()([x, skip])
  x = tfkl.Dropout(dropout)(x)

  out = residual_block(x, n_filters)
  return out

def encoder_block(x, n_filters, dropout):
  '''implementation of the encoder block described in
  https://www.sciencedirect.com/science/article/pii/S0169260721000201?via%3Dihub'''
  skip = residual_block(x, n_filters)
  x = tfkl.MaxPool2D(2)(skip)
  x = tfkl.Dropout(dropout)(x)
  return x, skip

def dilated_conv_block(x, filters):
  '''implementation of the bottleneck described in
  https://www.sciencedirect.com/science/article/pii/S0169260721000201?via%3Dihub'''
  for dilation_rate in [1, 2, 4, 8]:
      x = tfkl.Conv2D(filters, (3, 3), dilation_rate=dilation_rate, padding='same', activation='relu')(x)
  return x

def improved_unet(x, num_classes, dropout, n_filter):
  '''implementetion of the improved unet'''
  x = residual_block(x, n_filter)

  # encoder
  x, skip1 = encoder_block(x, n_filter*2, dropout)
  x, skip2 = encoder_block(x, n_filter*4, dropout)
  x, skip3 = encoder_block(x, n_filter*8, dropout)
  x, skip4 = encoder_block(x, n_filter*16, dropout)

  # bottleneck
  x = dilated_conv_block(x, n_filter*32)

  # decoder
  d1 = decoder_block(x, n_filter*16, skip4, dropout)
  d2 = decoder_block(d1, n_filter*8, skip3, dropout)
  d3 = decoder_block(d2, n_filter*4, skip2, dropout)
  d4 = decoder_block(d3, n_filter*2, skip1, dropout)

  resizing_layer = tf.keras.layers.Resizing(height=256, width=256, interpolation='bilinear')
  up_d1 = resizing_layer(d1)
  up_d2 = resizing_layer(d2)
  up_d3 = resizing_layer(d3)

  x = tfkl.Concatenate()([up_d1, up_d2, up_d3, d4])

  # head
  out = tfkl.Conv2D(num_classes, 1, padding="same", activation = "softmax")(x)

  return out

## Model Building & Callbacks

In [ ]:
#Build Model --> instantiate the weights of the defined architecture according to the choosen mode
def build_model(
    input_shape,
    num_classes = 7,
    learning_rate = 0.0001,
    dropout = 0.5,
    n_filter = 16,
    augmentation = None,
    seed = 42
):
    # Set the random seed for reproducibility
    tf.random.set_seed(seed)

    # Define the input layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Apply optional data augmentation
    if augmentation == None:
        x = input_layer
    else:
        x = augmentation(input_layer)

    output = improved_unet(x, NUM_CLASSES, dropout, n_filter)

    # Define the model
    model = tf.keras.Model(inputs=input_layer, outputs=output, name='Improved_UNET')

    return model

# compile model

In [ ]:
# lista con metriche per valutazione
metrics = [Mean_DICE(num_classes = 7)]

In [ ]:
# Configure the Adam optimizer
from pathlib import Path
adam_optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0001,  # Step size (α)
    beta_1=0.9,            # Exponential decay rate for the first moment (β1)
    beta_2=0.999,          # Exponential decay rate for the second moment (β2)
    epsilon=1e-8           # Small constant to prevent division by zero (ε)
)


#Create a directory tree to neatly save the models

path_output = '/kaggle/working/'
training_path = os.path.join(path_output, 'Training_' + datetime.now(gettz("Europe/Rome")).strftime("%Y-%m-%d-T%H:%M"))
checkpoint_path = os.path.join(training_path, 'Checkpoints')
best_model_path = os.path.join(training_path, 'Best_Model')

Path(training_path).mkdir(parents=True, exist_ok=True)
Path(checkpoint_path).mkdir(parents=True, exist_ok=True)
Path(best_model_path).mkdir(parents=True, exist_ok=True)

# callbacks and build and fit function
in this section the callback are initialized and a function that build the model and fit it is created. The latter is useful while computing the grid search because the hparams are given as input of the function.

In [ ]:
# callback for lr reducing
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10,
        verbose=1,
        min_lr=1e-10
    )

#Callback to keep track of the model with the best validation Mean_DICE
patience = 20
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # qui lui dice di monitorare il dice, però nel paper monitorano la val_loss. per ora userei quella, poi cambiamo
    mode='min',
    patience=patience,
    restore_best_weights=True
)

#Callback for the tensorboard monitoring
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=checkpoint_path,
    histogram_freq=1,  # Registra gli istogrammi dei pesi
    write_graph=False,  # Salva il grafo del modello
    write_images=False  # Salva immagini per visualizzazioni
)

In [ ]:
losses = ['combined: weighted and dice', 'dice_loss', 'focal_loss']

def build_and_fit(hparams):

  adam_optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0001,  # Step size (α)
    beta_1=0.9,            # Exponential decay rate for the first moment (β1)
    beta_2=0.999,          # Exponential decay rate for the second moment (β2)
    epsilon=1e-8           # Small constant to prevent division by zero (ε)
    )

  model = build_model(input_shape = (256, 256, 1),
                    num_classes = 7,
                    learning_rate = 0.0001,
                    dropout = hparams[HP_DROPOUT],
                    n_filter = hparams[HP_N_FILTERS],
                    augmentation = None,
                    seed = 42)

  # Get the loss function based on the hparams value
  loss_function = hparams[HP_LOSS]
  if loss_function == 'combined: weighted and dice':
    loss_function = combined_loss(num_classes=NUM_CLASSES, class_weights=normalized_weights, alpha=0.6, beta=0.4)
  elif loss_function == 'dice_loss':
    loss_function = dice_loss
  elif loss_function == 'focal_loss':
    loss_function = focal_loss
  else:
    raise ValueError(f"Invalid loss function: {loss_function}")

  model.compile(loss = loss_function,  # Use the resolved loss function
                  optimizer = adam_optimizer,
                  metrics = metrics)
  history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=100,
                    callbacks=[early_stopping, ReduceLROnPlateau, tensorboard_callback, hp.KerasCallback(hparams_path, hparams)])
  return history.history['val_Mean_DICE'][-1], model

## Model Training & Evaluation

# Creation of the hparam for tensorboard and set-up of the grid search

In [ ]:
from tensorboard.plugins.hparams import api as hp

hparams_path = os.path.join(checkpoint_path, 'hparams')
Path(hparams_path).mkdir(parents=True, exist_ok=True)

HP_DROPOUT = hp.HParam('dropout_rate', hp.RealInterval(0.1, 0.5))
HP_LOSS = hp.HParam('loss_function', hp.Discrete(['dice_loss', 'focal_loss','combined: weighted and dice']))
HP_N_FILTERS = hp.HParam('number_of_filters', hp.Discrete([8, 16]))
METRIC_DICE = 'val_mean_dice'

with tf.summary.create_file_writer(hparams_path).as_default():
  hp.hparams_config(
    hparams=[HP_DROPOUT, HP_LOSS, HP_N_FILTERS],
    metrics=[hp.Metric(METRIC_DICE, display_name='dice')],
  )

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    val_mean_dice, model = build_and_fit(hparams)
    tf.summary.scalar(METRIC_DICE, val_mean_dice, step=1)
  return model

# Model fit with different hparams

In [ ]:
!kill $(sudo lsof -t -i:6006)

In [ ]:
#model.fit --> actual training of the model using back-propagation for updating weights
#model.fit --> actual training of the model using back-propagation for updating weights
session_num = 0
for dropout in [0, 0.2, 0.5]:
  for i, loss in enumerate(losses):
    for n_filter in [8, 16]:
      print(f'dropout: {dropout}, loss: {loss}, n_filter: {n_filter}')
      hparams = {
          HP_DROPOUT: dropout,
          HP_LOSS: loss,
          HP_N_FILTERS: n_filter
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      model = run(hparams_path + '/' + run_name, hparams)
      model.save(f'dropout_{dropout}_loss_{i+1}_n_filter_{n_filter}.keras')
      session_num += 1
      del model
      tf.keras.backend.clear_session()

# Plotting the grid search in tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "$checkpoint_path"